<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

In [3]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IlRlYWNoIG1lIGEgbGVzc29uIG9uIHF1YWRyYXRpYyBlcXVhdGlvbnMuIEFzc3VtZSBJIGtub3cgYWJzb2x1dGVseSBub3RoaW5nIGFib3V0IGl0LiJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IiMjIFF1YWRyYXRpYyBFcXVhdGlvbnM6IEEgQmVnaW5uZXImIzM5O3MgR3VpZGVcXG5cXG5JbWFnaW5lIHlvdSYjMzk7cmUgdGhyb3dpbmcgYSBiYWxsIHVwIGluIHRoZSBhaXIuIEl0IGdvZXMgdXAsIHJlYWNoZXMgYSBwZWFrLCBhbmQgdGhlbiBmYWxscyBiYWNrIGRvd24uIFRoZSBwYXRoIGl0IHRha2VzIGNhbiBiZSBkZXNjcmliZWQgYnkgYSAqKnF1YWRyYXRpYyBlcXVhdGlvbioqLiBMZXQmIzM5O3MgZXhwbG9yZSB3aGF0IHRoYXQgbWVhbnM6XFxuXFxuKioxLiBXaGF0IGlzIGEgUXVhZHJhdGljIEVxdWF0aW9uPyoqXFxuXFxuSXQmIzM5O3MgYW4gZXF1YXRpb24gdGhhdCBsb29rcyBsaWtlIHRoaXM6ICoqYXjCsiArIGJ4ICsgYyA9IDAqKi4gRG9uJiMzOTt0IGJlIGludGltaWRhdGVkIGJ5IHRoZSBzeW1ib2xzISBMZXQmIzM5O3MgYnJlYWsgaXQgZG93bjpcXG5cXG4qICoqeCoqOiBUaGlzIHJlcHJlc2VudHMgYW4gdW5rbm93biB2YWx1ZSwgbGlrZSB0aGUgdGltZSBpdCB0YWtlcyBmb3IgdGhlIGJhbGwgdG8gcmVhY2ggdGhlIGdyb3VuZC5cXG4qICoqYSwgYiwgYyoqOiBUaGVzZSBhcmUgbnVtYmVycywgbGlrZSB0aGUgaW5pdGlhbCBzcGVlZCBvZiB0aGUgYmFsbCwgaXRzIHN0YXJ0aW5nIGhlaWdodCwgYW5kIHRoZSBlZmZlY3Qgb2YgZ3Jhdml0eS5cXG4qICoqwrIqKjogVGhpcyBtZWFucyAmcXVvdDtzcXVhcmVkLCZxdW90OyBvciBtdWx0aXBsaWVkIGJ5IGl0c2VsZi4gRm9yIGV4YW1wbGUsIHjCsiBpcyB0aGUgc2FtZSBhcyB4IHRpbWVzIHguXFxuXFxuKioyLiBXaHkgYXJlIHRoZXkgY2FsbGVkICZxdW90O1F1YWRyYXRpYyZxdW90Oz8qKlxcblxcblRoZSB3b3JkICZxdW90O3F1YWRyYXRpYyZxdW90OyBjb21lcyBmcm9tICZxdW90O3F1YWQsJnF1b3Q7IHdoaWNoIG1lYW5zICZxdW90O3NxdWFyZS4mcXVvdDsgVGhlIGhpZ2hlc3QgcG93ZXIgb2YgeCBpbiB0aGUgZXF1YXRpb24gaXMgMiwgaGVuY2UgdGhlIG5hbWUuXFxuXFxuKiozLiBXaGF0IGRvIHRoZXkgdGVsbCB1cz8qKlxcblxcblF1YWRyYXRpYyBlcXVhdGlvbnMgaGVscCB1cyBzb2x2ZSBwcm9ibGVtcyB3aGVyZSB0aGluZ3MgY2hhbmdlIGluIGEgbm9uLWxpbmVhciB3YXksIGxpa2UgdGhlIGJhbGwmIzM5O3MgdHJhamVjdG9yeS4gU29sdmluZyB0aGUgZXF1YXRpb24gbWVhbnMgZmluZGluZyB0aGUgdmFsdWVzIG9mIHggdGhhdCBtYWtlIHRoZSBlcXVhdGlvbiB0cnVlLiBUaGVzZSB2YWx1ZXMgYXJlIGNhbGxlZCAqKnJvb3RzKiogb3IgKipzb2x1dGlvbnMqKi5cXG5cXG4qKjQuIEhvdyBkbyB3ZSBzb2x2ZSB0aGVtPyoqXFxuXFxuVGhlcmUgYXJlIGRpZmZlcmVudCB3YXlzIHRvIHNvbHZlIHF1YWRyYXRpYyBlcXVhdGlvbnM6XFxuXFxuKiAqKkZhY3RvcmluZyoqOiBUaGlzIGludm9sdmVzIHJld3JpdGluZyB0aGUgZXF1YXRpb24gYXMgYSBwcm9kdWN0IG9mIHR3byBzaW1wbGVyIGV4cHJlc3Npb25zLlxcbiogKipRdWFkcmF0aWMgRm9ybXVsYSoqOiBUaGlzIGlzIGEgcG93ZXJmdWwgZm9ybXVsYSB0aGF0IGNhbiBzb2x2ZSBhbnkgcXVhZHJhdGljIGVxdWF0aW9uLiBJdCBsb29rcyBzY2FyeSwgYnV0IGl0JiMzOTtzIGp1c3QgYSByZWNpcGUgZm9yIGZpbmRpbmcgdGhlIHJvb3RzLlxcbiogKipHcmFwaGluZyoqOiBZb3UgY2FuIHBsb3QgdGhlIGVxdWF0aW9uIG9uIGEgZ3JhcGggYW5kIHNlZSB3aGVyZSBpdCBjcm9zc2VzIHRoZSB4LWF4aXMuIFRob3NlIHBvaW50cyBhcmUgdGhlIHNvbHV0aW9ucy5cXG5cXG4qKjUuIFdoeSBhcmUgdGhleSBpbXBvcnRhbnQ/KipcXG5cXG5RdWFkcmF0aWMgZXF1YXRpb25zIGhhdmUgYXBwbGljYXRpb25zIGluIHZhcmlvdXMgZmllbGRzOlxcblxcbiogKipQaHlzaWNzKio6IENhbGN1bGF0aW5nIHRoZSBtb3Rpb24gb2Ygb2JqZWN0cywgbGlrZSBvdXIgYmFsbCBleGFtcGxlLlxcbiogKipFbmdpbmVlcmluZyoqOiBEZXNpZ25pbmcgYnJpZGdlcywgYnVpbGRpbmdzLCBhbmQgb3RoZXIgc3RydWN0dXJlcy5cXG4qICoqRmluYW5jZSoqOiBQcmVkaWN0aW5nIG1hcmtldCB0cmVuZHMgYW5kIG1ha2luZyBpbnZlc3RtZW50IGRlY2lzaW9ucy5cXG4qICoqQ29tcHV0ZXIgU2NpZW5jZSoqOiBVc2VkIGluIGFsZ29yaXRobXMgYW5kIGNvbXB1dGVyIGdyYXBoaWNzLlxcblxcbioqNi4gTGV0JiMzOTtzIHNlZSBhbiBleGFtcGxlOioqXFxuXFxuSW1hZ2luZSB0aGUgZXF1YXRpb24gZGVzY3JpYmluZyB0aGUgYmFsbCYjMzk7cyBoZWlnaHQgaXM6ICoqaCA9IC01dMKyICsgMjB0ICsgMTAqKi4gSGVyZSwgaCBpcyB0aGUgaGVpZ2h0IGFuZCB0IGlzIHRoZSB0aW1lLiBTb2x2aW5nIHRoaXMgZXF1YXRpb24gd2lsbCB0ZWxsIHVzIHdoZW4gdGhlIGJhbGwgaGl0cyB0aGUgZ3JvdW5kIChoID0gMCkuIFdlIGNhbiB1c2UgdGhlIHF1YWRyYXRpYyBmb3JtdWxhIG9yIGdyYXBoIHRoZSBlcXVhdGlvbiB0byBmaW5kIHRoZSBzb2x1dGlvbnMuXFxuXFxuVGhpcyBpcyBqdXN0IGEgYmFzaWMgaW50cm9kdWN0aW9uIHRvIHF1YWRyYXRpYyBlcXVhdGlvbnMuIFRoZXJlJiMzOTtzIG11Y2ggbW9yZSB0byBsZWFybiwgYnV0IGhvcGVmdWxseSwgdGhpcyBnaXZlcyB5b3UgYSBzdGFydGluZyBwb2ludCB0byBleHBsb3JlIGZ1cnRoZXIhXFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJHaXZlIG1lIGFuIGVxdWF0aW9uIHRvIHNvbHZlIGFuZCB0ZXN0IG15IGtub3dsZWRnZSwgRG9uJ3QgZ2l2ZSBhbnN3ZXJzIGp1c3QgeWV0LiAifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiIqKlNvbHZlIHRoZSBmb2xsb3dpbmcgcXVhZHJhdGljIGVxdWF0aW9uOioqXFxuXFxuKip4wrIgLSA1eCArIDYgPSAwKipcXG5cXG5SZW1lbWJlciwgdGhlIHNvbHV0aW9ucyBhcmUgdGhlIHZhbHVlcyBvZiB4IHRoYXQgbWFrZSB0aGUgZXF1YXRpb24gdHJ1ZS4gWW91IGNhbiB1c2UgYW55IG1ldGhvZCB5b3UgbGlrZSwgc3VjaCBhcyBmYWN0b3JpbmcsIHRoZSBxdWFkcmF0aWMgZm9ybXVsYSwgb3IgZ3JhcGhpbmcuIEdvb2QgbHVjayEgXFxuIn1dfSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOlt7InRleHQiOiJ4PTAgYW5kIHg9MTAifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiJVbmZvcnR1bmF0ZWx5LCB4ID0gMCBhbmQgeCA9IDEwIGFyZSBub3Qgc29sdXRpb25zIHRvIHRoZSBlcXVhdGlvbiB4wrIgLSA1eCArIDYgPSAwLiBcXG5cXG5MZXQmIzM5O3MgY2hlY2s6XFxuXFxuKiBGb3IgeCA9IDA6ICgwKcKyIC0gNSgwKSArIDYgPSA2IOKJoCAwXFxuKiBGb3IgeCA9IDEwOiAoMTApwrIgLSA1KDEwKSArIDYgPSA1NiDiiaAgMFxcblxcbkFzIHlvdSBjYW4gc2VlLCBuZWl0aGVyIHZhbHVlIG1ha2VzIHRoZSBlcXVhdGlvbiB0cnVlLiBcXG5cXG5UcnkgdXNpbmcgYW5vdGhlciBtZXRob2QgbGlrZSBmYWN0b3Jpbmcgb3IgdGhlIHF1YWRyYXRpYyBmb3JtdWxhIHRvIGZpbmQgdGhlIGNvcnJlY3Qgc29sdXRpb25zLiBcXG4ifV19LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IlByb3ZpZGUgYSBkZXRhaWxlZCBtZXRob2QgdG8gc29sdmUgaXQifV19LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOlt7InRleHQiOiIjIyBTb2x2aW5nIHjCsiAtIDV4ICsgNiA9IDA6IEEgU3RlcC1ieS1TdGVwIEd1aWRlXFxuXFxuTGV0JiMzOTtzIHNvbHZlIHRoaXMgZXF1YXRpb24gdXNpbmcgdHdvIGRpZmZlcmVudCBtZXRob2RzOiBmYWN0b3JpbmcgYW5kIHRoZSBxdWFkcmF0aWMgZm9ybXVsYS5cXG5cXG4qKk1ldGhvZCAxOiBGYWN0b3JpbmcqKlxcblxcbjEuICoqSWRlbnRpZnkgdGhlIGZhY3RvcnMgb2YgdGhlIGNvbnN0YW50IHRlcm0gKGMpIHdob3NlIHN1bSBpcyB0aGUgY29lZmZpY2llbnQgb2YgdGhlIGxpbmVhciB0ZXJtIChiKS4qKiBcXG4gICAgKiBJbiB0aGlzIGNhc2UsIGMgPSA2IGFuZCBiID0gLTUuIFxcbiAgICAqIFdlIG5lZWQgdG8gZmluZCB0d28gbnVtYmVycyB0aGF0IG11bHRpcGx5IHRvIDYgYW5kIGFkZCB1cCB0byAtNS4gVGhlc2UgbnVtYmVycyBhcmUgLTIgYW5kIC0zLlxcblxcbjIuICoqUmV3cml0ZSB0aGUgbWlkZGxlIHRlcm0gdXNpbmcgdGhlIGlkZW50aWZpZWQgZmFjdG9ycy4qKlxcbiAgICAqIHjCsiAtIDV4ICsgNiBiZWNvbWVzIHjCsiAtIDJ4IC0gM3ggKyA2LlxcblxcbjMuICoqR3JvdXAgdGhlIHRlcm1zIGFuZCBmYWN0b3IgYnkgZ3JvdXBpbmcuKipcXG4gICAgKiAoeMKyIC0gMngpICsgKC0zeCArIDYpXFxuICAgICogeCh4IC0gMikgLSAzKHggLSAyKVxcblxcbjQuICoqRmFjdG9yIG91dCB0aGUgY29tbW9uIGJpbm9taWFsLioqXFxuICAgICogKHggLSAyKSh4IC0gMylcXG5cXG41LiAqKlNldCBlYWNoIGZhY3RvciBlcXVhbCB0byB6ZXJvIGFuZCBzb2x2ZSBmb3IgeC4qKlxcbiAgICAqIHggLSAyID0gMCAtLSZndDsgeCA9IDJcXG4gICAgKiB4IC0gMyA9IDAgLS0mZ3Q7IHggPSAzXFxuXFxuVGhlcmVmb3JlLCB0aGUgc29sdXRpb25zIHRvIHRoZSBlcXVhdGlvbiBhcmUgKip4ID0gMiBhbmQgeCA9IDMqKi5cXG5cXG4qKk1ldGhvZCAyOiBRdWFkcmF0aWMgRm9ybXVsYSoqXFxuXFxuVGhlIHF1YWRyYXRpYyBmb3JtdWxhIGlzIGEgcG93ZXJmdWwgdG9vbCB0aGF0IGNhbiBzb2x2ZSBhbnkgcXVhZHJhdGljIGVxdWF0aW9uLiBJdCBzdGF0ZXM6XFxuXFxuKip4ID0gKC1iIMKxIOKImihiwrIgLSA0YWMpKSAvICgyYSkqKlxcblxcbkZvciBvdXIgZXF1YXRpb24sIGEgPSAxLCBiID0gLTUsIGFuZCBjID0gNi4gUGx1Z2dpbmcgdGhlc2UgdmFsdWVzIGludG8gdGhlIGZvcm11bGE6XFxuXFxueCA9ICg1IMKxIOKImigoLTUpwrIgLSA0KDEpKDYpKSkgLyAoMigxKSlcXG54ID0gKDUgwrEg4oiaKDI1IC0gMjQpKSAvIDJcXG54ID0gKDUgwrEgMSkgLyAyXFxuXFxuVGhpcyBnaXZlcyB1cyB0d28gc29sdXRpb25zOlxcblxcbiogeCA9ICg1ICsgMSkgLyAyID0gM1xcbiogeCA9ICg1IC0gMSkgLyAyID0gMlxcblxcbkFnYWluLCB0aGUgc29sdXRpb25zIGFyZSAqKnggPSAyIGFuZCB4ID0gMyoqLlxcblxcbkJvdGggbWV0aG9kcyBsZWFkIHRvIHRoZSBzYW1lIHNvbHV0aW9ucy4gWW91IGNhbiBjaG9vc2Ugd2hpY2hldmVyIG1ldGhvZCB5b3UgZmluZCBtb3JlIGNvbWZvcnRhYmxlIG9yIGVmZmljaWVudCBmb3IgdGhlIHNwZWNpZmljIHByb2JsZW0geW91JiMzOTtyZSBzb2x2aW5nLlxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Teach me a lesson on quadratic equations. Assume I know absolutely nothing about it."
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "## Quadratic Equations: A Beginner&#39;s Guide\\n\\nImagine you&#39;re throwing a ball up in the air. It goes up, reaches a peak, and then falls back down. The path it takes can be described by a **quadratic equation**. Let&#39;s explore what that means:\\n\\n**1. What is a Quadratic Equation?**\\n\\nIt&#39;s an equation that looks like this: **ax\u00b2 + bx + c = 0**. Don&#39;t be intimidated by the symbols! Let&#39;s break it down:\\n\\n* **x**: This represents an unknown value, like the time it takes for the ball to reach the ground.\\n* **a, b, c**: These are numbers, like the initial speed of the ball, its starting height, and the effect of gravity.\\n* **\u00b2**: This means &quot;squared,

## Call `generate_content`

In [6]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

if response.candidates:
    display(Markdown(response.text))
else:
    # Print prompt feedback if available to understand the blocking reason
    if response.prompt_feedback:
        print(f"Prompt feedback: {response.prompt_feedback}")
    else:
        print("Error: No candidates returned. The prompt may have been blocked.")

Error: No candidates returned. The prompt may have been blocked.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [7]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Teach me a lesson on quadratic equations. Assume I know absolutely nothing about it. 

-------------------------------------------------------------------------------- 

Role: model 

## Quadratic Equations: A Beginner&#39;s Guide\n\nImagine you&#39;re throwing a ball up in the air. It goes up, reaches a peak, and then falls back down. The path it takes can be described by a **quadratic equation**. Let&#39;s explore what that means:\n\n**1. What is a Quadratic Equation?**\n\nIt&#39;s an equation that looks like this: **ax² + bx + c = 0**. Don&#39;t be intimidated by the symbols! Let&#39;s break it down:\n\n* **x**: This represents an unknown value, like the time it takes for the ball to reach the ground.\n* **a, b, c**: These are numbers, like the initial speed of the ball, its starting height, and the effect of gravity.\n* **²**: This means &quot;squared,&quot; or multiplied by itself. For example, x² is the same as x times x.\n\n**2. Why are they called &quot;Quadratic&q